# Numerical quadrature

## Newton-Cotes formulas

### Part a) Composite Simpson's rule

Write a function to calculate the composite Simpson's rule.

In [ ]:
def composite_Simpson(f, a, b, n):
    """
    Calculate the area under the function f between a and b with the composite Simpson's rule with n elements
    """
    integral = 0
    for i in range(n):
        xleft = a + i*(b-a)/n
        xright = xleft + (b-a)/n
        integral = integral + (xright - xleft)/6. * (f(xleft) + 4. * f((xleft+xright)/2) + f(xright))

    return integral

### Part b)

Test the accuracy for the integral
$$
\int_{-3}^5 \left(x^5 + 2x^4 - 9x^2 +13x\right) dx
$$ 

First define a function for the polynomial and a function for its derivative.

In [ ]:
def polynomial(x):
    f = x**5 + 2* x**4 - 9 * x**2 + 13*x
    return f

def polynomial_derivative(x):
    return 1/6 * x**6 + 2/5*x**5 - 9/3*x**3 + 13/2*x**2

Calculate the integral analytically and with the composite Simpson's rule for different numbers of intervals.

In [ ]:
# Exact integral
If_l = polynomial_derivative(-3)
If_u = polynomial_derivative(5)
I_analytical = (If_u - If_l)
print("The exact integral is:", I_analytical)

We calculate the analytical integral as
\begin{align*}
I_{ana} = \int_{-3}^5 \left(x^5 + 2x^4 - 9x^2 +13x\right) dx = \left[\frac{x^6}{6} + \frac{2x^5}{5} - 3x^3 + \frac{13x^2}{2}\right]_{-3}^5 = 3477.8\bar{6}
\end{align*}
which agrees with the value from the previous cell.

Now calculate the integral with the composite Simpson's rule and the numpy.trapz function for different numbers of intervals. Compare the results against the exact integral.

In [ ]:
# Calculate numerical integral for different numbers of intervals
import numpy as np
from IPython.display import display, Math

print("{:3s}, {:9s}, {:10s}, {:12s}, {:s}".format("N", "Simpson", "np.trapz", "e_Simpson", "e_trapz"))

I_Simpson = []
I_trapz = []
for i in range(8):
    N = 2**(i+1)
    I_Simpson.append(composite_Simpson(polynomial, -3, 5, N))
    x = np.linspace(-3, 5, N, endpoint=True)
    I_trapz.append(np.trapz(polynomial(x), x))
    print("{:3d}, {:9.4f}, {:10.4f}, {:e}, {:e}".format(N, I_Simpson[i], I_trapz[i], np.abs(I_Simpson[i] - I_analytical), np.abs(I_trapz[i] - I_analytical)))

The absolute error for the composite Simpson's rule reduces much quicker than for the composite trapezoidal rule, as is expected from their convergence orders.

### Part c) Double integrals

Now consider the double integral
$$
\int_{-2}^2\int_{0}^4 \left(x^2 - 3y^2 + xy^3\right) dxdy
$$

Calculate the analytical integral and compare this to the solution with your composite Simpson's rule function for different numbers of intervals.

Let us first calculate the analytical integral which is given by
\begin{align*}
I_{ana} &= \int_{-2}^2\int_{0}^4 \left(x^2 - 3y^2 + xy^3\right) dxdy 
 = \int_{-2}^2 \left[\frac{x^3}{3} - 3xy^2 + \frac{x^2y^3}{2}\right]_{0}^4 dy \\
 &=  \int_{-2}^2 \left(\frac{64}{3} - 12y^2 + 8 y^3\right)dy 
 = \left[\frac{64y}{3} - 4y^3 + 2y^4 \right]_{-2}^2 \\
 &= \frac{128}{3} - 32 + 32 - \left(\frac{-128}{3} + 32 + 32\right) = \frac{256}{3} - 64 \\
 &= \frac{64}{3} \approx 21.33
\end{align*}

First, we define the 2D function.

In [ ]:
def poly2(x, y):
    return x**2 - 3 * y**2 + x*y**3

Then we use the composite Simpson's rule twice to calculate the double integral. To do this we define a new function which calculates the inner integral for a fixed value of y.

In [ ]:
# Define a new function which fixes the second argument of poly2
def make_h(c):
   def h(x):
       return poly2(x, c)
   return h

# Integrate along the x direction for fixed y
def integrate_x(y):
    x_func = make_h(y) 
    return composite_Simpson(x_func, 0, 4, 5)

In [ ]:
I_2D = composite_Simpson(integrate_x, -2, 2, 5)
print(I_2D)

Running the script for different numbers of intervals shows that the Simpson's rule gives the exact result in this case (within the numerical accuracy of the computer). The Simpson's rule is accurate for polynomials of degree 3 or less which explains the exact result in this case.

## Gaussian quadrature

### Part a)

Consider the integral of
$$
f(x) = 3 - 12x + 3x^2 + 13x^3
$$
between the limits $a=1$ and $b=3$.

Approximate the integral 
$$
I_w = \int_a^b f(x) dx = \int_{-1}^1 f(\xi) d\xi \approx \sum_{j=0}^N w_j f(\xi_j)
$$
by Gaussian quadrature with two support points, i.e. $N=2$. For the two-point Gaussian quadrature the support points are $\xi_0=-\frac{1}{\sqrt{3}}\approx-0.5773503$ and $\xi_1=\frac{1}{\sqrt{3}}\approx0.5773503$ and the weights are $w_{1,2}=1$.

First we need to change the limits of integration to the limits $-1$ and $1$ so that you can use Gaussian quadrature.

A change of variable can be used to translate any definite integral onto the limits $-1$ and $1$. Define a new variable $\xi$ which is related to the original variable $x$ in the following way
$$
x = a_1 + a_2 \xi
$$
The lower limit $x=a$ should correspond to $\xi=-1$ and the upper limit $x=b$ should correspond to $\xi=1$ thus
\begin{align*}
a &= a_1 + a_2(-1) \\
b &= a_1 + a_2(1)
\end{align*}
This gives two equations for $a_1$ and $a_2$ which are
\begin{align*}
a_1 &= \frac{b+a}{2} \\
a_2 &= \frac{b-a}{2}
\end{align*}
With $a=1$ and $b=3$ we get
\begin{align*}
a_1 &= 2 \\
a_2 &= 1
\end{align*}
Now we can replace $x=2+\xi$ and $dx=d\xi$ to get
$$
I_w = \int_1^3 f(x) dx = \int_{-1}^1 f(2+\xi) d\xi 
$$

### Part b)

Calculate the integral and compare the value to the exact integral value.

In [ ]:
# Use lambda functions for the polynomial and its derivative
f = lambda x: 3 - 12*x + 3*x**2 + 13*x**3
If = lambda x: 3*x - 6*x**2 + x**3 + 13/4*x**4

# Quadrature weights
w = np.array([1, 1])

# Quadrature support points
xi = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])

# Gaussian quadrature and exact integral
I_Gaussian = np.sum(w @ f(2+xi))
I_analytical = If(3) - If(1)
print("The exact integral value is {} and the Gaussian quadrature give us {}.".format(I_analytical, I_Gaussian))

The integral with the translated limits 
$$
I_w = \int_1^3 f(x) dx = \int_{-1}^1 f(2+\xi) d\xi 
$$
can be calculated with Gaussian quadrature as
$$
I_w = \int_{-1}^1 f(2+\xi) d\xi = \sum_{i=0}^1 w_i f(2+\xi_i)=244
$$
where we have used the weights and support points given above.

In this case we can check the result by integrating the function analytically.
$$
\int_1^3 f(x) dx = \left[3x - 6x^2 + x^3 + \frac{13}{4}x^4\right]_{x=1}^{x=3} = 244
$$
Here the Gaussian quadrature gives the correct result. This was expected because Gaussian quadrature with 2 support points is accurate for polynomials of order up to and including 3 (2 times 2 minus 1).